# Wetterfrosch V1 Export

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.saved_model import signature_constants
from tensorflow.python.saved_model import tag_constants
from tensorflow.python.saved_model import builder
import os

/Users/markkeinhorster/.venv/tf_weather_service/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Das Modell wird mit Keras geladen

In [2]:
wetterfrosch_v1 = keras.models.load_model('../keras_models/wetterfrosch_v1.h5')

## Die Wetterfrosch V1 Signatur für TensorFlow-Serving wird zusammengebaut

In [3]:
tensor_info_input = tf.saved_model.utils.build_tensor_info(wetterfrosch_v1.input)
tensor_info_output = tf.saved_model.utils.build_tensor_info(wetterfrosch_v1.output)
prediction_signature = (
    tf.saved_model.signature_def_utils.build_signature_def(
        inputs={'input': tensor_info_input},
        outputs={'prediction': tensor_info_output},
        method_name=signature_constants.PREDICT_METHOD_NAME)
)

## Das Keras-Modell wird für TensorFlow-Serving exportiert

In [4]:
model_version = "1"
export_path = os.path.join("../models/weather", model_version)
tf_builder = builder.SavedModelBuilder(export_path)
with tf.keras.backend.get_session() as sess:
    tf_builder.add_meta_graph_and_variables(
        sess=sess,
        tags=[tag_constants.SERVING],
        signature_def_map={
            signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY: prediction_signature
            }
    )
    tf_builder.save()

INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: ../models/weather/1/saved_model.pb
